In [1]:
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.cluster import KMeans
from sklearn.neural_network import MLPRegressor # <--- This is your ANN (Neural Network)
from sklearn.metrics import mean_absolute_error, accuracy_score

In [3]:
# --- CONFIGURATION ---
DATA_PATH = "../data/neurohabit_synthetic_data.csv"
MODEL_SAVE_PATH = "../trained_models/"

In [4]:
# --- 1. LOAD DATA ---
df = pd.read_csv(DATA_PATH)
print(f"Data Loaded. Shape: {df.shape}")

Data Loaded. Shape: (1000, 5)


In [5]:
# Prepare Inputs (Features) and Outputs (Targets)
# We use these features to predict everything
features = ['sleep_hours', 'work_intensity', 'stress_level', 'mood_score']
X = df[features]
y_score = df['daily_performance_score']

In [6]:
# --- MODEL 1: THE PREDICTOR (Regression) ---
# Goal: Predict the exact Daily Score (0-100)
# Algo: ANN (Multi-Layer Perceptron) - Satisfies your "ANN" requirement
print("\n🧠 Training Model 1: The Predictor (ANN)...")

X_train, X_test, y_train, y_test = train_test_split(X, y_score, test_size=0.2, random_state=42)

# A simple Neural Network with 2 hidden layers (64 neurons, 32 neurons)
model_regressor = MLPRegressor(hidden_layer_sizes=(64, 32), max_iter=500, random_state=42)
model_regressor.fit(X_train, y_train)

# Test it
predictions = model_regressor.predict(X_test)
mae = mean_absolute_error(y_test, predictions)
print(f"   -> Accuracy Check: On average, the prediction is off by {mae:.2f} points.")
print("   -> (Lower is better. < 10 is great for FYP)")


🧠 Training Model 1: The Predictor (ANN)...
   -> Accuracy Check: On average, the prediction is off by 4.53 points.
   -> (Lower is better. < 10 is great for FYP)


In [7]:
# --- MODEL 2: THE TRIAGE (Classification) ---
# Goal: Classify day as 'High Performance' vs 'Maintenance Mode'
# Algo: Random Forest Classifier
print("\n🛡️ Training Model 2: The Triage (Classification)...")

# Create labels: If Score > 75, it's a "Peak" day. Else "Maintenance".
y_class = (df['daily_performance_score'] > 75).astype(int) 

rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(X, y_class)

acc = accuracy_score(y_class, rf_classifier.predict(X))
print(f"   -> Classification Accuracy: {acc*100:.1f}%")


🛡️ Training Model 2: The Triage (Classification)...
   -> Classification Accuracy: 100.0%


In [8]:
# --- MODEL 3: THE PERSONA (Clustering) ---
# Goal: Find patterns (e.g., "The Zombie" vs "The Machine")
# Algo: K-Means Clustering
print("\n🧩 Training Model 3: The Persona (Clustering)...")

kmeans = KMeans(n_clusters=3, random_state=42) # We want 3 personas
kmeans.fit(X)

# Assign the cluster back to the dataframe to see what they look like
df['cluster'] = kmeans.labels_
print("   -> Clusters defined. Here are the average stats for each Persona:")
print(df.groupby('cluster')[['sleep_hours', 'daily_performance_score']].mean())


🧩 Training Model 3: The Persona (Clustering)...
   -> Clusters defined. Here are the average stats for each Persona:
         sleep_hours  daily_performance_score
cluster                                      
0           8.314809                51.951985
1           7.072828                72.171739
2           5.889726                37.965087


In [9]:
# --- 4. SAVE THE BRAINS ---
print("\n💾 Saving Models to disk...")

joblib.dump(model_regressor, f"{MODEL_SAVE_PATH}neuro_predictor_ann.pkl")
joblib.dump(rf_classifier, f"{MODEL_SAVE_PATH}neuro_classifier.pkl")
joblib.dump(kmeans, f"{MODEL_SAVE_PATH}neuro_clusterer.pkl")

print(f"✅ DONE! 3 Models saved in {MODEL_SAVE_PATH}")


💾 Saving Models to disk...
✅ DONE! 3 Models saved in ../trained_models/
